In [1]:
from dfg_rating.model import factory
from dfg_rating.model.betting.betting import FixedBetting, BaseBetting
from dfg_rating.model.bookmaker.base_bookmaker import BaseBookmaker
from dfg_rating.model.evaluators.accuracy import RankProbabilityScore
from dfg_rating.model.evaluators.accuracy import Likelihood
from dfg_rating.model.evaluators.profitability import BettingReturnsEvaluator
from dfg_rating.model.forecast.base_forecast import SimpleForecast, BaseForecast
from dfg_rating.model.forecast.true_forecast import LogFunctionForecast
from dfg_rating.model.network.base_network import BaseNetwork

import networkx as nx
import pandas as pd
import numpy as np

from dfg_rating.model.network.simple_network import RoundRobinNetwork

pd.set_option('display.max_columns', None)
pd.options.display.width = None
pd.set_option('display.float_format', lambda x: '%.5f' % x)

In [2]:
network: BaseNetwork = factory.new_network(
    'multiple-round-robin',
    teams=18,
    days_between_rounds=7,
    seasons=10,
    league_teams=18,
    league_promotion=0,
    create=True
)
network.create_data()

Season 0
0
Relegation candidates {14: 2.0, 1: 6.0, 17: 10.0, 3: 15.0, 11: 17.0, 12: 17.0}
Relegation teams []
Season 1
1
Relegation candidates {1: 2.0, 11: 4.0, 14: 11.0, 17: 12.0, 4: 16.0, 8: 17.0}
Relegation teams []
Season 2
2
Relegation candidates {1: 3.0, 14: 7.0, 17: 8.0, 3: 10.0, 4: 11.0, 11: 18.0}
Relegation teams []
Season 3
3
Relegation candidates {1: 2.0, 14: 5.0, 4: 11.0, 17: 13.0, 15: 14.0, 16: 16.0}
Relegation teams []
Season 4
4
Relegation candidates {1: 1.0, 4: 8.0, 14: 9.0, 17: 10.0, 8: 11.0, 15: 19.0}
Relegation teams []
Season 5
5
Relegation candidates {1: 1.0, 14: 4.0, 3: 15.0, 4: 16.0, 8: 17.0, 16: 18.0}
Relegation teams []
Season 6
6
Relegation candidates {1: 1.0, 4: 8.0, 14: 8.0, 17: 8.0, 6: 13.0, 3: 17.0}
Relegation teams []
Season 7
7
Relegation candidates {1: 0.0, 17: 7.0, 4: 11.0, 6: 15.0, 11: 16.0, 14: 16.0}
Relegation teams []
Season 8
8
Relegation candidates {14: 2.0, 1: 3.0, 17: 7.0, 4: 11.0, 6: 14.0, 11: 15.0}
Relegation teams []
Season 9
9
Relegation ca

In [3]:
#network.print_data(schedule=True)

In [4]:
#adding biased bookmaker forecast
network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-1.2, 0], beta_parameter=0.006),
    forecast_name='bookmaker_forecast',
    base_ranking='true_rating'
)

#adding true forecast (reality)
network.add_forecast(
    forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[-0.9, 0.3], beta_parameter=0.006),
    forecast_name='true_forecast',
    base_ranking='true_rating'
)

None
None


In [5]:
bookmaker: BaseBookmaker = factory.new_bookmaker(
    'simple',
    error=factory.new_forecast_error(error_type='factor', error=0.0, scope='positive'),
    margin=factory.new_bookmaker_margin('simple', margin=0.05)
)
network.add_odds(
    bookmaker_name="bm",
    bookmaker=bookmaker,
    base_forecast='bookmaker_forecast'
)

In [6]:
betting = FixedBetting(1000)

In [7]:
def aggregate_measures(list_of_matches, prefix=''):
    return {prefix+'_rps': np.mean([m[3]['metrics']['rps'] for m in list_of_matches]), 
           prefix+'_likelihood': sum([m[3]['metrics']['likelihood'] for m in list_of_matches]), 
           prefix+'_betting_returns': sum([sum([b[0] for b in m[3]['metrics']['betting_returns']]) for m in list_of_matches]),
           prefix+'_betting_returns_expected': sum([sum(b[1] for b in m[3]['metrics']['betting_returns']) for m in list_of_matches])}

In [8]:
result_list = []
for c0 in np.arange(-2.0, 2.0, 0.5):
    for c1 in np.arange(c0, 2.0, 0.5):
        for beta in np.arange(0.0, 0.02, 0.005):
            
            print(f'Variables {c0}, {c1} and {beta}')
            
            rps = RankProbabilityScore(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
            betting_returns = BettingReturnsEvaluator(outcomes=['home', 'draw', 'away'], player_name = 'b', true_model = 'true_forecast', bookmaker_name = 'bm')
            likelihood = Likelihood(outcomes=['home', 'draw', 'away'], forecast_name = 'player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta))
            
            network.add_forecast(
            forecast=LogFunctionForecast(outcomes=['home', 'draw', 'away'], coefficients=[c0, c1], beta_parameter=beta),
            forecast_name='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta),
            base_ranking='true_rating'
            )
                
            network.add_bets(
            bettor_name='b',
            bookmaker='bm',
            betting=betting,
            base_forecast='player_forecast'+'_'+str(c0)+'_'+str(c1)+'_'+str(beta)
            )
            
            network.add_evaluation(rps, 'rps')
            network.add_evaluation(betting_returns, 'betting_returns')
            network.add_evaluation(likelihood, 'likelihood')
        
            is_matches = [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] < 4]
            oos_matches = [(a,h, match_id, match_attributes) for a,h, match_id, match_attributes in network.iterate_over_games() if match_attributes['season'] > 3]
            
            result_is = aggregate_measures(is_matches, 'is')
            result_oos = aggregate_measures(oos_matches, 'oos')
            result = {
                'c0' : c0, 'c1' : c1, 'beta' : beta, **result_is, **result_oos
            }
            
            result_list.append(result)
            
        network.serialize_network('economic_network')
            
            
            

            

Variables -2.0, -2.0 and 0.0
None


c:\users\dell-pc\documents\projekte\dfg-rating\dfg_rating\model\evaluators\accuracy.py:45: RuntimeWarning: divide by zero encountered in log
  for m, o in zip(model, observed) if o > 0


Variables -2.0, -2.0 and 0.005
None
Variables -2.0, -2.0 and 0.01
None
Variables -2.0, -2.0 and 0.015
None
Variables -2.0, -1.5 and 0.0
None
Variables -2.0, -1.5 and 0.005
None
Variables -2.0, -1.5 and 0.01
None
Variables -2.0, -1.5 and 0.015
None
Variables -2.0, -1.0 and 0.0
None
Variables -2.0, -1.0 and 0.005
None
Variables -2.0, -1.0 and 0.01
None
Variables -2.0, -1.0 and 0.015
None
Variables -2.0, -0.5 and 0.0
None
Variables -2.0, -0.5 and 0.005
None
Variables -2.0, -0.5 and 0.01
None
Variables -2.0, -0.5 and 0.015
None
Variables -2.0, 0.0 and 0.0
None
Variables -2.0, 0.0 and 0.005
None
Variables -2.0, 0.0 and 0.01
None
Variables -2.0, 0.0 and 0.015
None
Variables -2.0, 0.5 and 0.0
None
Variables -2.0, 0.5 and 0.005
None
Variables -2.0, 0.5 and 0.01
None
Variables -2.0, 0.5 and 0.015
None
Variables -2.0, 1.0 and 0.0
None
Variables -2.0, 1.0 and 0.005
None
Variables -2.0, 1.0 and 0.01
None
Variables -2.0, 1.0 and 0.015
None
Variables -2.0, 1.5 and 0.0
None
Variables -2.0, 1.5 and 0.

In [9]:
df = pd.DataFrame(result_list)
df.to_excel("Results_Economics.xlsx")

In [21]:
for a,h,m_id,attributes in network.iterate_over_games():
    print([v for v in attributes['metrics']['betting_returns']])

[(0.0, -0.0), (-0.0, -0.0), (-10.0, 25.277985139861414)]
[(0.0, -0.0), (-0.0, -0.0), (-10.0, 24.272249946487825)]
[(0.0, -0.0), (-0.0, -0.0), (-10.0, 25.27212012856845)]
[(-0.0, -0.0), (-0.0, -0.0), (43.77123340774681, 25.357233152781543)]
[(-0.0, -0.0), (0.0, -0.0), (-10.0, 25.36119974424116)]
[(0.0, -0.0), (-0.0, -0.0), (-10.0, 25.302072580210297)]
[(0.0, -0.0), (-0.0, -0.0), (-10.0, 25.36884389448199)]
[(-0.0, -0.0), (-0.0, -0.0), (38.80234520350971, 25.19229849970038)]
[(-0.0, -0.0), (0.0, -0.0), (-10.0, 24.755599411291655)]
[(-0.0, -0.0), (-0.0, -0.0), (31.1432749055661, 23.587953812088962)]
[(-0.0, -0.0), (-0.0, -0.0), (45.528230633984464, 25.277985139861414)]
[(-0.0, -0.0), (-0.0, -0.0), (53.21354234746744, 24.272249946487825)]
[(-0.0, -0.0), (0.0, -0.0), (-10.0, 25.27212012856845)]
[(0.0, -0.0), (-0.0, -0.0), (-10.0, 25.357233152781543)]
[(-0.0, -0.0), (0.0, -0.0), (-10.0, 25.36119974424116)]
[(-0.0, -0.0), (0.0, -0.0), (-10.0, 25.302072580210297)]
[(-0.0, -0.0), (-0.0, -0.0), 